In [2]:
from services.database import Database
from services.dataloading import *
Database.initialize()

# Get all historical SPY tickers that are also in the Tiingo DB

In [2]:
#get tickers that are in tiingo and GFD
ticker_info = Database.query("SELECT b.* from ticker_index_membership b inner join tiingo_tickers a on a.ticker = b.ticker where b.equity_index = 'S&P500' and b.endDate = '2023-04-21 00:00:00.000000' and b.startDate < '2010-01-01 00:00:00.000000'")
ticker_info

,id,source,ticker,equity_index,priceCurrency,startDate,endDate
0,4605119990101,gfd,A,S&P500,USD,1999-01-01 00:00:00.000000,2023-04-21 00:00:00.000000
1,4838120050101,gfd,AAL,S&P500,USD,2005-01-01 00:00:00.000000,2023-04-21 00:00:00.000000
2,4923120010101,gfd,AAP,S&P500,USD,2001-01-01 00:00:00.000000,2023-04-21 00:00:00.000000
3,4923120010101,gfd,AAP,S&P500,USD,2001-01-01 00:00:00.000000,2023-04-21 00:00:00.000000
4,4934119800101,gfd,AAPL,S&P500,USD,1980-01-01 00:00:00.000000,2023-04-21 00:00:00.000000
...,...,...,...,...,...,...,...
666,107563119360101,gfd,XRX,S&P500,USD,1936-01-01 00:00:00.000000,2023-04-21 00:00:00.000000
667,108039119970101,gfd,YUM,S&P500,USD,1997-01-01 00:00:00.000000,2023-04-21 00:00:00.000000
668,108106120010101,gfd,ZBH,S&P500,USD,2001-01-01 00:00:00.000000,2023-04-21 00:00:00.000000
669,108116119910101,gfd,ZBRA,S&P500,USD,1991-01-01 00:00:00.000000,2023-04-21 00:00:00.000000


In [4]:
#find a valid time period for the assets
startDate = ticker_info.startDate.max()
endDate = ticker_info.endDate.min()
ticker_info = ticker_info[['ticker', 'priceCurrency']]
ticker_info['ticker_currency'] = ticker_info.ticker + "-" + ticker_info.priceCurrency

C:\Users\david\AppData\Local\Temp\ipykernel_5548\3669705573.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_info['ticker_currency'] = ticker_info.ticker + "-" + ticker_info.priceCurrency


In [5]:
startDate

'2009-01-01 00:00:00.000000'

In [6]:
endDate

'2023-04-21 00:00:00.000000'

In [ ]:
#alpha vantage
df_av = retrieve_alpha_vantage(freq='W', ticker_info=ticker_info)
merged_df_av = merge_equities_df(Database, df_av)

In [ ]:
#yahoo finance
df_yf =retrieve_yahoo_finance(startDate, endDate, freq='D', ticker_info=(ticker_info))
merged_df_yf = merge_equities_df(Database, df_yf)

In [ ]:
#tiingo
df_tiingo = retrieve_tiingo(startDate, endDate, ticker_info)
merged_df_tiingo = merge_equities_df(Database, df_tiingo)

In [184]:
df_prices = retrieve_ml_factor()
merged_ml_factor = merge_equities_df(Database, df_prices)

In [51]:
add_dfs_to_db('equities_series', Database, [merged_df_tiingo, merged_df_yf, merged_df_av, merged_ml_factor], indices_of_primary_key=[0])

# Loop through all tickers historically included in SPY via tiingo and yahoo and load them

In [6]:
ticker_info = Database.query("SELECT * from ticker_index_membership where equity_index = 'S&P500' and startDate <> '1900-01-01 00:00:00.000000'")
ticker_info

,id,source,ticker,equity_index,priceCurrency,startDate,endDate
0,132118970101,gfd,KSU2,S&P500,USD,1897-01-01 00:00:00.000000,2021-12-13 00:00:00.000000
1,147119960101,gfd,BRK.B,S&P500,USD,1996-01-01 00:00:00.000000,2023-04-21 00:00:00.000000
2,160119590101,gfd,BF.B,S&P500,USD,1959-01-01 00:00:00.000000,2023-04-21 00:00:00.000000
3,173119860101,gfd,CERN2,S&P500,USD,1986-01-01 00:00:00.000000,2022-06-08 00:00:00.000000
4,282119900101,gfd,XLNX1,S&P500,USD,1990-01-01 00:00:00.000000,2022-02-14 00:00:00.000000
...,...,...,...,...,...,...,...
1791,108039119970101,gfd,YUM,S&P500,USD,1997-01-01 00:00:00.000000,2023-04-21 00:00:00.000000
1792,108106120010101,gfd,ZBH,S&P500,USD,2001-01-01 00:00:00.000000,2023-04-21 00:00:00.000000
1793,108116119910101,gfd,ZBRA,S&P500,USD,1991-01-01 00:00:00.000000,2023-04-21 00:00:00.000000
1794,108269119660101,gfd,ZION,S&P500,USD,1966-01-01 00:00:00.000000,2023-04-21 00:00:00.000000


In [8]:
i = 1
for name, group in ticker_info.groupby('ticker'):
    #find a valid time period for the assets
    startDate = group.startDate.min()
    endDate = group.endDate.max()
    group = group.iloc[:1]
    group = group[['ticker', 'priceCurrency']]
    group['ticker_currency'] = group.ticker + "-" + group.priceCurrency

    try:
        df_yf =retrieve_yahoo_finance(startDate, endDate, freq='D', ticker_info=group)
        merged_df_yf = merge_equities_df(Database, df_yf)
    except:
        print("yahoo failed... len of df: ", len(df_yf))

    try:
        df_tiingo = []
        df_tiingo = retrieve_tiingo(startDate, endDate, group)
        merged_df_tiingo = merge_equities_df(Database, df_tiingo)
    except:
        print("tiingo failed... len of df: ", len(df_tiingo))

    try:
        add_dfs_to_db('equities_series', Database, [merged_df_tiingo, merged_df_yf], indices_of_primary_key=[0])
        print("Added "+ str(i) + " stocks to the db")
        print(group)
        print("*"*20)
        i+=1
    except:
        print("adding "+name + " failed!")

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 2023-03-01 00:00:00: No timezone found, symbol may be delisted
yahoo failed... len of df:  0
tiingo failed... len of df:  0
adding 2023-03-01 00:00:00 failed!
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 2023-05-01 00:00:00: No timezone found, symbol may be delisted
yahoo failed... len of df:  0
tiingo failed... len of df:  0
adding 2023-05-01 00:00:00 failed!
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 2023-09-01 00:00:00: No timezone found, symbol may be delisted
yahoo failed... len of df:  0
tiingo failed... len of df:  0
adding 2023-09-01 00:00:00 failed!
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 2023-12-01 00:00:00: No timezone found, symbol may be delisted
yahoo failed... len of df:  0
tiingo failed... len of df:  0
adding 2023-12-01 00:00:00 failed!


C:\Users\david\anaconda3\envs\svmmvo\lib\site-packages\pandas_datareader\tiingo.py:234: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  return pd.concat(dfs, self._concat_axis)


Added 1 stocks to the db
    ticker priceCurrency ticker_currency
994      A           USD           A-USD
********************
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- A4: No timezone found, symbol may be delisted
yahoo failed... len of df:  0
tiingo failed... len of df:  0
Added 2 stocks to the db
    ticker priceCurrency ticker_currency
365     A4           USD          A4-USD
********************
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AABA1: No timezone found, symbol may be delisted
yahoo failed... len of df:  0
tiingo failed... len of df:  0
Added 3 stocks to the db
  ticker priceCurrency ticker_currency
8  AABA1           USD       AABA1-USD
********************
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAF3: No timezone found, symbol may be delisted
yahoo failed... len of df:  0
tiingo failed... len of df:  0
Added 4 stocks to the

C:\Users\david\anaconda3\envs\svmmvo\lib\site-packages\pandas_datareader\tiingo.py:234: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  return pd.concat(dfs, self._concat_axis)


Added 5 stocks to the db
    ticker priceCurrency ticker_currency
995    AAL           USD         AAL-USD
********************
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAL1: No timezone found, symbol may be delisted
yahoo failed... len of df:  0
tiingo failed... len of df:  0
Added 6 stocks to the db
    ticker priceCurrency ticker_currency
182   AAL1           USD        AAL1-USD
********************
[*********************100%***********************]  1 of 1 completed


...

Added 1788 stocks to the db
     ticker priceCurrency ticker_currency
1793   ZBRA           USD        ZBRA-USD
********************
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ZETHQ1: No timezone found, symbol may be delisted
yahoo failed... len of df:  0
tiingo failed... len of df:  0
Added 1789 stocks to the db
     ticker priceCurrency ticker_currency
505  ZETHQ1           USD      ZETHQ1-USD
********************
[*********************100%***********************]  1 of 1 completed


C:\Users\david\anaconda3\envs\svmmvo\lib\site-packages\pandas_datareader\tiingo.py:234: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  return pd.concat(dfs, self._concat_axis)


Added 1790 stocks to the db
     ticker priceCurrency ticker_currency
1794   ZION           USD        ZION-USD
********************
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ZRN1: No timezone found, symbol may be delisted
yahoo failed... len of df:  0
tiingo failed... len of df:  0
Added 1791 stocks to the db
    ticker priceCurrency ticker_currency
625   ZRN1           USD        ZRN1-USD
********************
[*********************100%***********************]  1 of 1 completed


C:\Users\david\anaconda3\envs\svmmvo\lib\site-packages\pandas_datareader\tiingo.py:234: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  return pd.concat(dfs, self._concat_axis)


Added 1792 stocks to the db
     ticker priceCurrency ticker_currency
1795    ZTS           USD         ZTS-USD
********************
